In [7]:
"""
test_rag.py - 멀티-레이어 RAG 파이프라인 통합 테스트
====================================================
sample_data → router → retriever → chain 전체 흐름 테스트
"""

import os
import sys

from dotenv import load_dotenv

# .env에서 API 키 로드
load_dotenv()

# ==========================================
# 모듈 임포트
# ==========================================
from sample_data import get_all_documents
from router import build_router, resolve_filters, apply_airline_filter
from retriever import filter_documents, build_retriever
from chain import format_docs_with_source, build_qa_chain, build_danger_chain


# ==========================================
# 전역 초기화
# ==========================================
print("=" * 60)
print("  ✈️  항공 보안 RAG 챗봇 - 멀티레이어 파이프라인 테스트")
print("=" * 60)

ALL_DOCS = get_all_documents()
print(f"\n📦 전체 문서 로드 완료: {len(ALL_DOCS)}건")

intent_chain = build_router()
qa_chain = build_qa_chain()
danger_chain = build_danger_chain()

print("✅ 라우터 및 체인 초기화 완료!\n")


# ==========================================
# 메인 처리 함수
# ==========================================
def process_chat(user_query: str) -> str:
    """사용자 질문을 받아 전체 파이프라인을 실행합니다."""
    print(f"\n🗣️  사용자 질문: {user_query}")
    print("-" * 55)

    # ── Step 1: 의도 분석 ──
    intent = intent_chain.invoke({"query": user_query})
    print(
        f"🔍 [라우터] 위험물:{intent.is_dangerous} | "
        f"노선:{intent.flight_type} | 수하물:{intent.baggage_type} | "
        f"카테고리:{intent.category} | 항공사:{intent.airline}"
    )

    # ── Step 2: 위험물 즉시 차단 ──
    if intent.is_dangerous:
        # 위험물 관련 문서 검색 후 경고 답변 생성
        danger_filters = {"category": "general", "flight_type": ["common"]}
        retriever = build_retriever(ALL_DOCS, danger_filters, k=2)
        docs = retriever.invoke(user_query)
        context = format_docs_with_source(docs)
        answer = danger_chain.invoke({"context": context, "question": user_query})
        return f"🚨 {answer}"

    # ── Step 3: 필수 정보 부족 시 되묻기 ──
    missing_info = []
    if not intent.flight_type:
        missing_info.append("국내선/국제선 여부")
    if not intent.baggage_type:
        missing_info.append("기내 반입/위탁 여부")

    if missing_info:
        return (
            f"💬 정확한 규정 확인을 위해 "
            f"{'와(과) '.join(missing_info)}를 먼저 알려주시겠어요?"
        )

    # ── Step 4: 메타데이터 필터 구성 ──
    filters = resolve_filters(intent)
    print(f"🏷️  [필터] {filters}")

    # 항공사 특수 필터 처리
    working_docs = apply_airline_filter(ALL_DOCS, filters)

    # ── Step 5: 하이브리드 검색 ──
    print("📥 [검색] 하이브리드 리트리버 가동 중...")
    retriever = build_retriever(working_docs, filters, k=3)
    retrieved_docs = retriever.invoke(user_query)

    print(f"📄 [검색 결과] {len(retrieved_docs)}건 검색됨")
    for i, doc in enumerate(retrieved_docs, 1):
        src = doc.metadata.get("source", "?")
        cat = doc.metadata.get("category", "?")
        ft = doc.metadata.get("flight_type", "?")
        print(f"   #{i} source={src} | category={cat} | flight_type={ft}")

    # ── Step 6: LLM 답변 생성 ──
    print("🤖 [답변 생성] LLM이 문서를 읽고 답변을 작성 중...")
    context = format_docs_with_source(retrieved_docs)
    answer = qa_chain.invoke({"context": context, "question": user_query})

    return answer


# ==========================================
# 테스트 실행
# ==========================================
def run_tests():
    """6개 시나리오를 순차 실행합니다."""
    tests = [
        {
            "name": "정보 부족 → 되묻기",
            "query": "고추장 가져가도 돼?",
        },
        {
            "name": "위험물 → 즉시 차단",
            "query": "부탄가스를 가방에 넣어도 돼?",
        },
        {
            "name": "국제선 + 액체류 필터링",
            "query": "국제선 기내에 화장품 가져가도 될까요?",
        },
        {
            "name": "항공사(KE) + 배터리 필터링",
            "query": "대한항공 타는데 보조배터리 위탁 수하물로 부칠 수 있나요?",
        },
        {
            "name": "국내선 필터링",
            "query": "국내선 타는데 음료수 기내에 들고 탈 수 있어?",
        },
        {
            "name": "보안검색 후 면세점 음료",
            "query": "국제선인데 보안검색대 지나서 면세점에서 산 음료수 기내에 가져갈 수 있나요?",
        },
    ]

    for i, test in enumerate(tests, 1):
        print(f"\n{'=' * 60}")
        print(f"  📋 테스트 {i}/{len(tests)}: {test['name']}")
        print(f"{'=' * 60}")

        answer = process_chat(test["query"])
        print(f"\n👉 챗봇 답변:\n{answer}")

    print(f"\n{'=' * 60}")
    print("  ✅ 모든 테스트 완료!")
    print(f"{'=' * 60}")


if __name__ == "__main__":
    run_tests()


  ✈️  항공 보안 RAG 챗봇 - 멀티레이어 파이프라인 테스트

📦 전체 문서 로드 완료: 16건
✅ 라우터 및 체인 초기화 완료!


  📋 테스트 1/6: 정보 부족 → 되묻기

🗣️  사용자 질문: 고추장 가져가도 돼?
-------------------------------------------------------
🔍 [라우터] 위험물:False | 노선:None | 수하물:None | 카테고리:food | 항공사:None

👉 챗봇 답변:
💬 정확한 규정 확인을 위해 국내선/국제선 여부와(과) 기내 반입/위탁 여부를 먼저 알려주시겠어요?

  📋 테스트 2/6: 위험물 → 즉시 차단

🗣️  사용자 질문: 부탄가스를 가방에 넣어도 돼?
-------------------------------------------------------
🔍 [라우터] 위험물:True | 노선:None | 수하물:위탁 | 카테고리:general | 항공사:None
📂 검색 대상 문서: 3건 (전체 16건 중)

👉 챗봇 답변:
🚨 부탄가스는 항공 보안 규정에 따라 기내 및 위탁수하물 모두 반입이 금지된 위험물입니다. 따라서 가방에 넣어도 절대 반입할 수 없습니다. 

관련 규정에 따르면, 국토부 고시 [p.8]에서 인화성 물질인 부탄가스는 기내 반입 및 위탁수하물 모두 금지된다고 명시되어 있습니다. 안전한 비행을 위해 이러한 규정을 반드시 준수해 주시기 바랍니다.

  📋 테스트 3/6: 국제선 + 액체류 필터링

🗣️  사용자 질문: 국제선 기내에 화장품 가져가도 될까요?
-------------------------------------------------------
🔍 [라우터] 위험물:False | 노선:국제선 | 수하물:기내 | 카테고리:liquid | 항공사:None
🏷️  [필터] {'flight_type': ['international', 'common'], 'category': 'liquid'}
📥 [검색] 하이브리드 리트리버 가동 중...
📂 검색

In [9]:
import os
from typing import Optional, List
from pydantic import BaseModel, Field
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.chat_message_histories import ChatMessageHistory

# 1. 초기화
load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 의도 분석 스키마 (메모리를 통해 채워질 구조)
class FlightIntent(BaseModel):
    is_dangerous: bool = Field(default=False)
    item_name: Optional[str] = Field(default=None, description="물품 명칭 (예: 고추장, 술, 향수)")
    flight_type: Optional[str] = Field(default=None, description="'국내선' 또는 '국제선'")
    baggage_type: Optional[str] = Field(default=None, description="'기내' 또는 '위탁'")

# 3. 지식 베이스 (면세점 규정 추가)
def get_updated_knowledge_base():
    return [
        Document(
            page_content="국제선 기내 액체류 제한: 100ml 이하 용기만 가능, 1L 지퍼백 1개에 담아야 함.",
            metadata={"scope": "국제선", "baggage_type": "기내", "source": "보안규정 제5조"}
        ),
        Document(
            page_content="면세점 액체류 특례(STEB): 보안검색대 통과 후 면세점에서 구입한 주류, 향수 등 액체류는 '액체물 보안봉투(STEB)'에 영수증과 함께 봉인된 경우 100ml를 초과해도 기내 반입이 가능합니다. 최종 목적지까지 봉인을 뜯어서는 안 됩니다.",
            metadata={"scope": "국제선", "baggage_type": "기내", "source": "면세품 반입 규정"}
        ),
        Document(
            page_content="국내선 액체류: 국내선은 액체류 기내 반입에 용량 제한이 없습니다.",
            metadata={"scope": "국내선", "baggage_type": "기내", "source": "국내선 가이드"}
        )
    ]

# 4. 멀티턴 RAG 클래스
class MultiTurnAviationBot:
    def __init__(self):
        self.all_docs = get_updated_knowledge_base()
        self.history = ChatMessageHistory() # 세션별 관리가 필요하면 dict로 확장 가능
        
        # 의도 분석용 프롬프트 (대화 기록 참조)
        self.router_prompt = ChatPromptTemplate.from_messages([
            ("system", "이전 대화와 현재 질문을 참고하여 사용자의 의도를 분석하세요. 정보가 없으면 None으로 둡니다."),
            MessagesPlaceholder(variable_name="history"),
            ("user", "{query}")
        ])
        self.router = self.router_prompt | llm.with_structured_output(FlightIntent)

        self.qa_chain = ChatPromptTemplate.from_messages([
            ("system", "공항 보안 전문가입니다. [규정]에 근거해 답하세요.\n\n[규정]\n{context}"),
            MessagesPlaceholder(variable_name="history"),
            ("user", "{question}")
        ]) | llm | StrOutputParser()

    def run(self, user_input: str):
        # 1. 의도 분석 (이전 대화 기록 포함)
        intent = self.router.invoke({"query": user_input, "history": self.history.messages})
        
        # 2. 위험물 선제 차단
        if intent.is_dangerous:
            return "🚨 해당 물품은 반입 금지 위험물입니다."

        # 3. 필수 정보(노선/수하물) 부족 시 질문 및 메모리 저장
        if not intent.flight_type or not intent.baggage_type or not intent.item_name:
            self.history.add_user_message(user_input)
            
            missing = []
            if not intent.item_name: missing.append("어떤 물건")
            if not intent.flight_type: missing.append("국내선/국제선 여부")
            if not intent.baggage_type: missing.append("기내/위탁 여부")
            
            response = f"💬 정확한 확인을 위해 {'와(과) '.join(missing)}을(를) 알려주시겠어요?"
            self.history.add_ai_message(response)
            return response

        # 4. 필터링된 검색 (면세점 키워드가 있으면 STEB 규정 우선 검색)
        filtered_docs = [d for d in self.all_docs if d.metadata["scope"] in [intent.flight_type, "common"]]
        
        # 하이브리드 리트리버 가동
        bm25 = BM25Retriever.from_documents(filtered_docs)
        vectorstore = FAISS.from_documents(filtered_docs, embeddings)
        ensemble = EnsembleRetriever(retrievers=[bm25, vectorstore.as_retriever()], weights=[0.5, 0.5])
        
        context_docs = ensemble.invoke(user_input)
        context_text = "\n".join([f"[{d.metadata['source']}] {d.page_content}" for d in context_docs])

        # 5. 최종 답변 생성 및 메모리 업데이트
        final_answer = self.qa_chain.invoke({
            "context": context_text,
            "question": user_input,
            "history": self.history.messages
        })
        
        self.history.add_user_message(user_input)
        self.history.add_ai_message(final_answer)
        return final_answer

In [14]:
def interactive_test():
    # 1. 봇 초기화
    bot = MultiTurnAviationBot()
    
    print("="*60)
    print("✈️  항공 보안 검색 RAG 챗봇 테스트 모드")
    print("대화를 종료하려면 '종료', 'exit', 'q'를 입력하세요.")
    print("="*60)

    while True:
        # 2. 사용자 입력 받기
        user_input = input("\n👤 사용자: ").strip()
        
        # 종료 조건
        if user_input.lower() in ['종료', 'exit', 'q', 'quit']:
            print("\n👋 테스트를 종료합니다. 즐거운 여행 되세요!")
            break
            
        if not user_input:
            continue

        try:
            # 3. 챗봇 엔진 실행
            response = bot.run(user_input)
            
            # 4. 결과 출력
            print(f"\n🤖 챗봇: {response}")
            
        except Exception as e:
            print(f"\n❌ 에러 발생: {e}")
            print("API 키 설정이나 라이브러리 설치 상태를 확인해주세요.")

# 실행
if __name__ == "__main__":
    interactive_test()

✈️  항공 보안 검색 RAG 챗봇 테스트 모드
대화를 종료하려면 '종료', 'exit', 'q'를 입력하세요.

🤖 챗봇: 💬 정확한 확인을 위해 어떤 물건와(과) 국내선/국제선 여부와(과) 기내/위탁 여부을(를) 알려주시겠어요?

🤖 챗봇: 💬 정확한 확인을 위해 어떤 물건와(과) 기내/위탁 여부을(를) 알려주시겠어요?

🤖 챗봇: 라이터는 국제선 비행기 기내에 반입할 수 없습니다. 보안 규정에 따라 라이터는 위탁 수하물로도 반입할 수 없으며, 기내에 반입할 수 있는 물품이 아닙니다. 따라서 라이터는 여행 전에 반드시 처분하시거나 다른 방법으로 안전하게 보관하시기 바랍니다.

👋 테스트를 종료합니다. 즐거운 여행 되세요!
